# Exploration of the deterministic BusSim model

In [ ]:
from BusSim_deterministic import run_model
from IPython.display import HTML    # for animation
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mogp_emulator

# Increase default figure size
plt.rcParams["figure.figsize"] = [12, 6]
plt.rcParams["font.size"] = 16

## Set up the model

We'll run the model with a sensible set of default parameters, as used in earlier simulations in this project.
The two that we will investigate in more detail are **traffic speed** and **arrival rate**.

**Traffic speed** does what it says - it gives the speed of the buses when they travel between the stops.


**Arrival rate** is more complex.
In the default case, it is constant at each stop over time.
However, the stops have different values, set randomly at the beginning of the simulation.
It determines how many people wait at each stop.
When we investigate the parameter more thoroughly further down in this notebook we will use a different (although still constant with time) rate at each stop; for now, we use the same rate at all stops.

In [ ]:
def run_model_w_params(ArrivalRate=None, TrafficSpeed=14, bus_ids_out=[0, 1, 2], seed=123, maxDemand=2, DEBUG=False):
    # Hold most parameters constant, apart from the two (TrafficSpeed and ArrivalRate) that we will investigate in this notebook.
    # Outputs are the times at which the buses of the provided IDs (bus_ids_out) reach their final destinations.
        
    if seed is not None:
        np.random.seed(seed) # Make departure rates consistent across all simulations
    
    NumberOfStop = 20
    minDemand = 0.5
     
    # Initialise the remaining model parameters
    model_params = {
        "dt": 10,
        "minDemand": minDemand,        
        "NumberOfStop": NumberOfStop,
        "LengthBetweenStop": 2000,
        "EndTime": 6000,
        "Headway": 5 * 60,
        "BurnIn": 1 * 60,
        "AlightTime": 1,
        "BoardTime": 3,
        "StoppingTime": 3,
        "BusAcceleration": 3
    }
    
    if ArrivalRate is None:
        ArrivalRate = np.random.uniform(minDemand / 60, maxDemand / 60, NumberOfStop) 
    else:
        ArrivalRate = ArrivalRate * np.ones(NumberOfStop)
        
    DepartureRate = np.sort(np.random.uniform(0.05, 0.5, NumberOfStop)) # Sorted as more passengers get off near the end of the route

    # Run the model with all plots turned off (some still produced as default; we turn them off with %%capture later)
    model, model_params, ArrivalData, StateData, GroundTruth, GPSData = \
        run_model(model_params, TrafficSpeed, ArrivalRate, DepartureRate, False, False, False, True)
        
    bus_pos = np.array([bus.trajectory for bus in model.buses])    
    time = np.arange(0, model.EndTime, model.dt)
    total_distance = model_params["NumberOfStop"] * model_params["LengthBetweenStop"]

    bus_t_end = [time[np.argmax(bp>=total_distance)] for bp in bus_pos]
    bus_end_times = [bus_t_end[id] for id in bus_ids_out]
    
    if DEBUG:
        print(bus_t_end)
    
    return bus_end_times, GPSData

## Vary traffic speed

Perform a parameter sweep over one variable, the traffic speed.
We'll capture a few variations on the same output: the time at which the buses listed in `ts_bus_ids` reaches their final destination.
(Later on, we'll see if we can predict those times using a GP.)

In [ ]:
%%capture

ts_bus_ids = [0, 1, 2, 3, 6, 9]

traffic_speeds = np.arange(15, 50, 2.5)
ts_bus_end_times = np.zeros((len(traffic_speeds), len(ts_bus_ids)))

for ind, ts in enumerate(traffic_speeds):
    bus_end_t, _ = run_model_w_params(TrafficSpeed=ts, bus_ids_out=ts_bus_ids)
    ts_bus_end_times[ind,:] = bus_end_t

In [ ]:
plt.plot(traffic_speeds, ts_bus_end_times, marker=".", linestyle="")
plt.xlabel("Traffic speed")
plt.ylabel("Time at final destination")
plt.legend(labels=["bus " + str(id) for id in ts_bus_ids], loc="right")
plt.show()

Most of the buses in this investigation have a relatively consistent relationship between final arrival time and traffic speed: at low speeds, they take longer and as speed increases they reach their destination faster, up until a point where increasing the speed has very little further effect.

The first buses to depart show more interesting variation with traffic speed, where at some (especially higher) traffic speeds the second bus overtakes the first after catching up with it at a stop.

## Vary the maximum demand

We'll now try a more complex case - we'll vary how busy the bus service is by altering the maximum demand.

Demand at the individual stops was set randomly in the previous example, but was held constant at each stop over time as the traffic speed was varied.

We'll first try using the same demand at each stop, and later on try randomised demand levels as before, with a varying upper limit.

In [ ]:
%%capture
# Prevent plots from showing - uncalibrated setting generates one per model

md_bus_ids = [0, 1, 2, 3, 4, 5, 6]

max_demand = np.arange(0.25, 5.0, 0.25)
md_bus_end_times = np.zeros((len(max_demand), len(md_bus_ids)))


for ind, max_dem in enumerate(max_demand): 
    print("Maximum demand: ", max_dem)
    
    ArrivalRate = max_dem/60
    
    bus_end_t, _ = run_model_w_params(ArrivalRate, bus_ids_out=md_bus_ids)
    md_bus_end_times[ind,:] = bus_end_t

In [ ]:
plt.plot(max_demand, md_bus_end_times, marker=".", linestyle="")
plt.xlabel("Maximum (constant) demand")
plt.ylabel("Time at final destination")
plt.legend(labels=["bus " + str(id) for id in md_bus_ids], loc="lower right")
plt.show()

## Using GPs to emulate time at final destination based on traffic speed

We'll try to emulate relatively simple outputs at first, sticking to only a few for each of our investigations.
In the first attempt at using the emulator, we'll attempt to emulate the time that buses 0, 4 and 9 (or the buses listed in `ts_bus_ids`, if altered) arrive at their final destination for different values of traffic speed.

We'll provide fewer input points than we show in the plot above, and see how it performs.

In [ ]:
%%capture
lhd = mogp_emulator.LatinHypercubeDesign([(15.0, 50.0)])

n_simulations = 4
lhd_traffic_speeds = lhd.sample(n_simulations)
print(lhd_traffic_speeds)

ts_simulation_output = np.zeros((len(ts_bus_ids), n_simulations))

for ind, ts in enumerate(lhd_traffic_speeds):
    sim_out, _ = run_model_w_params(TrafficSpeed=ts[0], bus_ids_out=ts_bus_ids)
    ts_simulation_output[:,ind] = np.array(sim_out)

### Fit the GP

In [ ]:
ts_gp = mogp_emulator.MultiOutputGP(lhd_traffic_speeds, ts_simulation_output)
ts_gp = mogp_emulator.fit_GP_MAP(ts_gp)

### Test out the GP

In [ ]:
%%capture
n_validation = 50

validation_traffic_speeds = lhd.sample(n_validation)

predicted_ts = ts_gp.predict(validation_traffic_speeds)

actual_ts = np.zeros((len(ts_bus_ids), n_validation))
for ind, ts in enumerate(validation_traffic_speeds):
    sim_out, _ = run_model_w_params(TrafficSpeed=ts[0], bus_ids_out=ts_bus_ids)
    actual_ts[:,ind] = np.array(sim_out)

In [ ]:
# Set the markers used in the plot
plt_mk = { "train": "o",      # Used to train the GP
           "pred": "+",       # Predicted by the GP
           "sim": "x" }       # Same TS as "pred", full simulation
plt_mk_legend = [matplotlib.lines.Line2D([0], [0], marker=plt_mk["train"], color="w", label="Training",
                 markerfacecolor="k", markeredgecolor="k"), 
                 matplotlib.lines.Line2D([0], [0], marker=plt_mk["pred"], color="w", label="Predicted (GP)",
                 markerfacecolor="k", markeredgecolor="k"), 
                 matplotlib.lines.Line2D([0], [0], marker=plt_mk["sim"], color="w", label="Simulated",
                 markerfacecolor="k", markeredgecolor="k")]

In [ ]:
legend_entries = []

for ind, (a, p, u) in enumerate(zip(actual_ts, predicted_ts.mean, predicted_ts.unc)):
    plt.plot(validation_traffic_speeds, a, marker=plt_mk["sim"], linestyle="", color=plt.cm.Set1(ind))
    plt.plot(validation_traffic_speeds, p, marker=plt_mk["pred"], linestyle="", color=plt.cm.Set1(ind))
    plt.plot(lhd_traffic_speeds, ts_simulation_output[ind,], marker=plt_mk["train"], linestyle="", color=plt.cm.Set1(ind))
    
    legend_entries.append(matplotlib.patches.Patch(facecolor=plt.cm.Set1(ind), edgecolor=None, label="Bus {}".format(ts_bus_ids[ind])))
    
    o = np.argsort(validation_traffic_speeds, axis=None)
    plt.fill_between(validation_traffic_speeds.flatten()[o], p[o]-u[o], p[o]+u[o],
                     color=plt.cm.Set1(ind), alpha=0.2, linewidth=0)
            
plt.xlabel("Traffic speed")
plt.ylabel("Time at final destination")
plt.legend(handles=legend_entries+plt_mk_legend, loc="upper left", bbox_to_anchor=(1.05, 1))
plt.show()

Generally, the GP is doing a good job of simulating the final arrival time of each bus.

The uncertainty in the prediction of the arrival time of the first bus is particularly interesting, as the bounds are much wider than those of the other buses.
Around speeds of 40-45, bus 0 is overtaken by buses 1 and 2, so this increase in uncertainty does make sense.

## Using GPs to emulate time at final destination based on maximum demand

In [ ]:
%%capture
md_lhd = mogp_emulator.LatinHypercubeDesign([(0.25, 5.0)])

md_bus_ids = [0, 1, 2, 6]
n_simulations = 8
md_lhd_sample = md_lhd.sample(n_simulations)

md_simulation_output = np.zeros((len(md_bus_ids), n_simulations))

for ind, max_dem in enumerate(md_lhd_sample):
    ArrivalRate = max_dem[0]/60
    sim_out, _ = run_model_w_params(ArrivalRate=ArrivalRate, bus_ids_out=md_bus_ids)
    md_simulation_output[:,ind] = np.array(sim_out)

In [ ]:
md_gp = mogp_emulator.MultiOutputGP(md_lhd_sample, md_simulation_output)
md_gp = mogp_emulator.fit_GP_MAP(md_gp)

In [ ]:
%%capture
n_validation = 50

md_validation = md_lhd.sample(n_validation)

md_predicted = md_gp.predict(md_validation)

md_actual = np.zeros((len(md_bus_ids), n_validation))
for ind, max_dem in enumerate(md_validation):
    ArrivalRate = max_dem[0]/60
    sim_out, _ = run_model_w_params(ArrivalRate=ArrivalRate, bus_ids_out=md_bus_ids)
    md_actual[:,ind] = np.array(sim_out)

In [ ]:
legend_entries = []

for ind, (a, p, u) in enumerate(zip(md_actual, md_predicted.mean, md_predicted.unc)):
    plt.plot(md_validation, a, marker=plt_mk["sim"], linestyle="", color=plt.cm.Set1(ind))
    plt.plot(md_validation, p, marker=plt_mk["pred"], linestyle="", color=plt.cm.Set1(ind))
    plt.plot(md_lhd_sample, md_simulation_output[ind,], marker=plt_mk["train"], linestyle="", color=plt.cm.Set1(ind))
    
    legend_entries.append(matplotlib.patches.Patch(facecolor=plt.cm.Set1(ind), edgecolor=None, label="Bus {}".format(md_bus_ids[ind])))
   
    o = np.argsort(md_validation, axis=None)
    plt.fill_between(md_validation.flatten()[o], p[o]-u[o], p[o]+u[o],
                     color=plt.cm.Set1(ind), alpha=0.2, linewidth=0)
        
plt.xlabel("Max demand")
plt.ylabel("Time at final destination")
plt.legend(handles=legend_entries+plt_mk_legend, loc="upper left", bbox_to_anchor=(1.05, 1))
plt.ylim((0, 10000))
plt.show()

## Using a GP to estimate arrival time at final destination based on two parameters

Essentially we'll try the same procedure as before, but this time with two variable parameters.
We're looking out for how difficult it is for the GP to still provide a good estimate given that coverage of the parameter space is now going to be far harder.

As before, we run a set of simulations to train our GP on.
We'll take 12 samples from a Latin hypercube to pull out samples which cover the parameter space of traffic speeds of 15-50 and maximum demand at each bus stop of 0.25-5.
We then fit the GP and test its performance via comparison of a further set of simulations against corresponding samples from the GP.

In [ ]:
%%capture
two_lhd = mogp_emulator.LatinHypercubeDesign([(15.0, 50.0), (0.25, 5.0)])

two_bus_ids = [0, 1, 2, 6]
n_simulations = 12

two_train_par = two_lhd.sample(n_simulations)
two_train_out = np.zeros((len(two_bus_ids), n_simulations))

for ind, par in enumerate(two_train_par):
    sim_out, _ = run_model_w_params(ArrivalRate=par[1]/60, TrafficSpeed=par[0], bus_ids_out=two_bus_ids)
    two_train_out[:,ind] = np.array(sim_out)

In [ ]:
two_gp = mogp_emulator.MultiOutputGP(two_train_par, two_train_out)
two_gp = mogp_emulator.fit_GP_MAP(two_gp)

In [ ]:
%%capture
n_validation = 20

two_validation_par = two_lhd.sample(n_validation)
two_validation_pred = two_gp.predict(two_validation_par)

two_validation_actual = np.zeros((len(two_bus_ids), n_validation))
for ind, par in enumerate(two_validation_par):
    sim_out, _ = run_model_w_params(ArrivalRate=par[1]/60, TrafficSpeed=par[0], bus_ids_out=two_bus_ids)
    two_validation_actual[:,ind] = np.array(sim_out)

Visualising how well the GP performs is trickier now that we are in two dimensions.
First, we'll print out the predicted and simulated final arrival times for one of the buses for a quick manual comparison.

In [ ]:
bus_index = 3
print("Bus %i\n-----" % two_bus_ids[bus_index])

print("traffic speed\tmax demand\tprediction\t\tactual")
for par, mean, unc, act in zip(two_validation_par, two_validation_pred.mean.T, two_validation_pred.unc.T, two_validation_actual.T):
    print("%.2f\t\t%.2f\t\t%.2f +/- %.2f\t%.2f" % (par[0], par[1], mean[bus_index], unc[bus_index], act[bus_index]))

In [ ]:
bus_index = 1

fig_train = plt.figure()
plt.scatter(two_train_par[:,1], two_train_par[:,0], c=two_train_out[bus_index,:], marker="o")

plt.xlabel("Max demand")
plt.ylabel("Traffic speed")
plt.colorbar(label="Arrival time (Bus {})".format(two_bus_ids[bus_index]))
xlims = (0, 5)
plt.xlim(xlims)
plt.show()

### Fix constant traffic speed for validation

In [ ]:
%%capture
ts_fixed = 40
two_validation_par_fix = two_validation_par.copy()
two_validation_par_fix[:,0] = ts_fixed

two_validation_pred_fix = two_gp.predict(two_validation_par_fix)

two_validation_actual_fix = np.zeros((len(two_bus_ids), n_validation))
for ind, par in enumerate(two_validation_par_fix):
    sim_out, _ = run_model_w_params(ArrivalRate=par[1]/60, TrafficSpeed=ts_fixed, bus_ids_out=two_bus_ids)
    two_validation_actual_fix[:,ind] = np.array(sim_out)

In [ ]:
legend_entries = []

for ind, (a, p, u) in enumerate(zip(two_validation_actual_fix, two_validation_pred_fix.mean, two_validation_pred_fix.unc)):
    plt.plot(two_validation_par_fix[:,1], a, marker=plt_mk["sim"], linestyle="", color=plt.cm.Set1(ind))
    plt.plot(two_validation_par_fix[:,1], p, marker=plt_mk["pred"], linestyle="", color=plt.cm.Set1(ind))
    # Note that training data hasn't been included here as it was obtained at different values of traffic speed    
    
    legend_entries.append(matplotlib.patches.Patch(facecolor=plt.cm.Set1(ind), edgecolor=None, label="Bus {}".format(md_bus_ids[ind])))
        
    o = np.argsort(two_validation_par_fix[:,1], axis=None)
    plt.fill_between(two_validation_par_fix[o,1], p[o]-u[o], p[o]+u[o],
                     color=plt.cm.Set1(ind), alpha=0.2, linewidth=0)
        
plt.xlabel("Max demand")
plt.ylabel("Time at final destination")
plt.title("Fixed traffic speed of {}".format(ts_fixed))
plt.legend(handles=legend_entries+plt_mk_legend[1:], loc="upper left", bbox_to_anchor=(1.05, 1))
plt.ylim((0, 7000))
plt.show()

Note that the parameter combinations used to fit the GP may not have had traffic speed set to the fixed value above.

For instance, the traffic speed used in the plot above is near the following points used to train the GP:

In [ ]:
fig_train.gca().plot(xlims, [ts_fixed, ts_fixed], color="k", linestyle="--")
fig_train

### Testing the impact of additional points to train the GP

It is clear from the plots above that regions of parameter space without adequate coverage when fitting the GP result in high uncertainties when making predictions using the GP.
Interestingly, the mean predictions from the GP are very close to the outputs of the corresponding simulations despite this issue.

In [ ]:
%%capture
n_consimulations = [8, 16, 32, 64]

twocon_gps = []

# Fit a GP with each number of input simulations
for n in n_consimulations:
    twocon_train_par = two_lhd.sample(n)    # Use the LHD from earlier, with limits of 15-50 for traffic speed and 0.25-5 for arrival rate
    twocon_train_out = np.zeros((len(two_bus_ids), n))

    for ind, par in enumerate(twocon_train_par):
        sim_out, _ = run_model_w_params(ArrivalRate=par[1]/60, TrafficSpeed=par[0], bus_ids_out=two_bus_ids)
        twocon_train_out[:,ind] = np.array(sim_out)
        
    twocon_gp = mogp_emulator.MultiOutputGP(twocon_train_par, twocon_train_out)
    twocon_gp = mogp_emulator.fit_GP_MAP(twocon_gp)
    twocon_gps.append(twocon_gp)

In [ ]:
# Generate the parameter combinations at which we want to test out the GP
n_validation = 10
twocon_validation_par = two_lhd.sample(n_validation)

# Use each GP to predict the arrival times given these parameters
twocon_predictions = []
for gp in twocon_gps:
    twocon_predictions.append(gp.predict(twocon_validation_par))

In [ ]:
%%capture
# Perform the full simulation at each of these parameter combinations
twocon_validation_actual = np.zeros((len(two_bus_ids), n_validation))
twocon_validation_gps = []
for ind, par in enumerate(twocon_validation_par):
    sim_out, gps = run_model_w_params(ArrivalRate=par[1]/60, TrafficSpeed=par[0], bus_ids_out=two_bus_ids)
    twocon_validation_actual[:,ind] = np.array(sim_out)
    twocon_validation_gps.append(gps)

In [ ]:
fig, axes = plt.subplots(*twocon_predictions[0].mean.shape, sharex=True, sharey=True)

diffs = np.zeros((len(n_consimulations), twocon_predictions[0].mean.shape[0], twocon_predictions[0].mean.shape[1]))
for p_ind, preds in enumerate(twocon_predictions):
    diffs[p_ind,:, :] = np.abs(preds.mean - twocon_validation_actual)
    
for bus_ind in range(twocon_predictions[0].mean.shape[0]):
    for par_ind in range(twocon_predictions[0].mean.shape[1]):
        axes[bus_ind, par_ind].scatter(n_consimulations, diffs[:, bus_ind, par_ind], marker="o")
        
        if bus_ind == twocon_predictions[0].mean.shape[0] - 1:
            axes[bus_ind, par_ind].set(xlabel=f"TS={twocon_validation_par[par_ind,0]:.1f}" + "\n" + f"AR={twocon_validation_par[par_ind,1]:.1f}")
    
        if par_ind == 0:
            axes[bus_ind, par_ind].set(ylabel="Bus " + str(two_bus_ids[bus_ind]))

        
fig.supxlabel("Number of parameter pairs used to train GP", y=-0.075);
fig.supylabel("Difference in final arrival time", x=-0.075);

#### Look into parameter combinations where GP emulates data less well

In [ ]:
par_index = 3
n_train_ind = 2
bus_ind_plot = 2
plt.plot(two_bus_ids, twocon_predictions[2].mean[:,3], "o", label="Predictions using GP")
plt.plot(two_bus_ids, twocon_validation_actual[:,3], "o", label="Simulation")
plt.legend(loc="upper left", bbox_to_anchor=(1.05, 1))
plt.xlabel("Bus ID")
plt.ylabel("Time to reach final stop")
plt.show()
print("Input points:\t", n_consimulations[n_train_ind])
print("Input parameters: ")
print("\tTraffic speed:\t", twocon_validation_par[par_index, 0])
print("\tArrival rate:\t", twocon_validation_par[par_index, 1])

In [ ]:
twocon_predictions[0].mean

In [ ]:
# Set up plot at first time-step
fig, ax = plt.subplots()
bus_loc_plot, = ax.plot(twocon_validation_gps[par_index][0,:], range(len(twocon_validation_gps[par_index][0,:])), "o")
plt.xlim((0, 40000))
plt.xlabel("Distance / m")
plt.ylabel("Bus ID")
ax.set_title("Time = 0")
plt.show()

In [ ]:
def drawframe(n, locations, points, axes):
    axes.set_title("Time = {}".format(n*10))
    points.set_xdata(locations[n])
    return points, axes

In [ ]:
anim = matplotlib.animation.FuncAnimation(fig, drawframe, frames=len(gps), interval=50, fargs=(gps, bus_loc_plot, ax), blit=False)
plt.show()

In [ ]:
html = anim.to_html5_video()
HTML(html)

In [ ]:
# twocon_validation_actual[:,par_ind]
len(twocon_predictions)

# sim_out, gps = run_model_w_params(ArrivalRate=twocon_validation_par[par_index, 1]/60, TrafficSpeed=twocon_validation_par[par_index, 0], bus_ids_out=two_bus_ids, DEBUG=True)

In [ ]:
fig, axes = plt.subplots(*twocon_predictions[0].mean.shape, sharex=True, sharey=True)

diffs = np.zeros((len(n_consimulations), twocon_predictions[0].unc.shape[0], twocon_predictions[0].unc.shape[1]))
for p_ind, preds in enumerate(twocon_predictions):
    diffs[p_ind,:, :] = preds.unc
    
for bus_ind in range(twocon_predictions[0].unc.shape[0]):
    for par_ind in range(twocon_predictions[0].unc.shape[1]):
        axes[bus_ind, par_ind].scatter(n_consimulations, diffs[:, bus_ind, par_ind], marker="o")
        
        if bus_ind == twocon_predictions[0].mean.shape[0] - 1:
            axes[bus_ind, par_ind].set(xlabel=f"TS={twocon_validation_par[par_ind,0]:.1f}" + "\n" + f"AR={twocon_validation_par[par_ind,1]:.1f}")
    
        if par_ind == 0:
            axes[bus_ind, par_ind].set(ylabel="Bus " + str(two_bus_ids[bus_ind]))

        
fig.supxlabel("Number of parameter pairs used to train GP", y=-0.075);
fig.supylabel("Uncertainty in arrival time", x=-0.075);

plt.ylim((0, 5000))

- Visualise exact bus movements (GUI)
- Check adding more training points to one-parameter versions
- More plots of uncertainties associated with each GP
- Pop scatter plots back into notebook
- Are largest uncertinties associated with occasions where buses swap order?
- Is there a param which decreases occurences of bus swapping?

- Decision to make: have we got a sensible set of parameters to work with? Then move to stochastic model

- **Links for Jon**

In [ ]:
%%capture
end_times, gps = run_model_w_params()

In [ ]:
gps[300]

In [ ]:
# Set up plot at first time-step
fig, ax = plt.subplots()
bus_loc_plot, = ax.plot(gps[0], range(len(gps[0])), "o")
plt.xlim((0, 40000))
plt.xlabel("Distance / m")
plt.ylabel("Bus ID")
plt.show()

In [ ]:
def drawframe(n, locations, points):
    points.set_xdata(locations[n])
    return points,

In [ ]:
from matplotlib import animation
anim = animation.FuncAnimation(fig, drawframe, frames=len(gps), interval=50, fargs=(gps, bus_loc_plot), blit=False)
plt.show()

In [ ]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)